<a href="https://colab.research.google.com/github/dmarx/notebooks/blob/text2video-modelscope/modelscope_text2video_w_hf_diffusers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers huggingface_hub accelerate
!pip install git+https://github.com/huggingface/diffusers/

In [ ]:
# https://github.com/huggingface/diffusers/pull/2738

# import torch
# from diffusers import TextToVideoSDPipelineOutput, DPMSolverMultistepScheduler
# from diffusers.utils import export_to_video

# pipe = TextToVideoSDPipelineOutput.from_pretrained("diffusers/ms-text-to-video-sd", torch_dtype=torch.float16)
# pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
# pipe = pipe.to("cuda")

# prompt = "Spiderman is surfing"
# video_frames = pipe(prompt).frames
# video_path = export_to_video(video_frames)
# print(video_path)

In [ ]:
# https://github.com/huggingface/diffusers/pull/2738
import torch
from diffusers import DiffusionPipeline
from diffusers.utils import export_to_video
pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
pipe = pipe.to("cuda")
prompt = "Spiderman is surfing"
video_frames = pipe(prompt).frames
video_path = export_to_video(video_frames)
video_path

In [ ]:
from IPython.display import HTML
from base64 import b64encode

!cp {video_path} /content/tmp.mp4
mp4 = open('/content/tmp.mp4','rb').read()

decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=400 controls><source src="{decoded_vid}" type="video/mp4"></video>')